In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from pathlib import Path

### Comment

**Generally**
- each file is 2s of data
- activity name and time stamp (start?) is on the file name

**Data inside**
- shape of (3,) 
- first element has 5 values - Acc ( Time stamp, sensor Time, X ,Y ,Z)
- 2nd,3rd elements is gyroscope and barometer

np.stack

[ [A],
  [G],
  [B]
      ]



### Testing

In [ ]:
test1 = np.load("../raw_data/fall_clean/fall/fall_ASUS_X013DB_1606911589384.npy")
test2 = np.load("../raw_data/fall_clean/fall/fall_SM_G960F_1606746602242.npy")


In [ ]:
list_np = []
list_np.append(test1[0])
list_np.append(test2[0])

In [ ]:
np.array(list_np)[1].shape

### Train preparation

In [2]:
import os
fall_path = "../raw_data/yann_data/dataset_2020-11-29-no-fall/fall_data/"
file_list = os.listdir(fall_path)

In [3]:
file_list.remove(".DS_Store")

In [4]:
file_list

['fall']

In [ ]:
list_np = []
list_c = []
file_list = ['climb-stairs', 'stay', 'down-stairs', 'walk', 'cycle', 'run']
print("start")
for file in file_list:
    if file == ".DS_Store":
        continue
    path = os.path.join(fall_path,file)
    npy_file = os.listdir(path)
    category = file
    print(f"start {file}")
    for npy in npy_file:
        if npy == ".DS_Store":
            continue
        file_path = os.path.join(path,npy)
        value = np.load(file_path, allow_pickle=True)[0][:,[2,3,4,]]
        list_np.append(value)
        category_int = file_list.index(category)
        list_c.append(category_int)
    print(f"end {file}")
np.savez("val.npz", value = np.array(list_np), cat = np.array(list_c))


In [5]:
list_np = []
list_c = []
file_list = ['fall']
print("start")
for file in file_list:
    if file == ".DS_Store":
        continue
    path = os.path.join(fall_path,file)
    npy_file = os.listdir(path)
    category = file
    print(f"start {file}")
    for npy in npy_file:
        if npy == ".DS_Store":
            continue
        file_path = os.path.join(path,npy)
        value = np.load(file_path, allow_pickle=True)[0][:,[2,3,4,]]
        list_np.append(value)
        category_int = file_list.index(category)
        list_c.append(6)
    print(f"end {file}")
np.savez("fall.npz", value = np.array(list_np), cat = np.array(list_c))

start
start fall
end fall


### Testing around

In [ ]:
test_path = "../raw_data/yann_data/dataset_2020-11-29-no-fall/test.npz"
train_path = "../raw_data/yann_data/dataset_2020-11-29-no-fall/train.npz"

In [ ]:
npzfile_train = np.load(train_path,allow_pickle=True)
npzfile_train.files

In [ ]:
npzfile_train["value"].shape

In [ ]:
npzfile_test = np.load(test_path,allow_pickle=True)
npzfile_test.files

In [ ]:
npzfile_test["value"].shape

In [ ]:
np.savez("fallguys.npz", train_value = npzfile_train["value"], train_cat = npzfile_train["cat"],
        test_value = npzfile_test["value"], test_cat = npzfile_test["cat"])

### Train

In [ ]:
# load file

DATA_URL = '../raw_data/yann_data/dataset_2020-11-29-no-fall/fallguys.npz'

with np.load(DATA_URL, allow_pickle=True) as data:
  train_examples = data['train_value']
  train_labels = data['train_cat']
  test_examples = data['test_value']
  test_labels = data['test_cat']

In [ ]:
# datat structure is different
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    train_examples, padding="post", dtype='float', value=-999.,
)

In [ ]:
padded_outputs = tf.keras.preprocessing.sequence.pad_sequences(
    test_examples, padding="post", dtype='float32', value=-999.,
)

In [ ]:
# label to data
train_dataset = tf.data.Dataset.from_tensor_slices((padded_inputs, train_labels))

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((padded_outputs, test_labels))

In [ ]:
# shuffle batch size 

BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Masking(mask_value=-999.))
# Add a LSTM layer with 64 internal units.
model.add(tf.keras.layers.LSTM(64))

model.add(tf.keras.layers.Dense(7,activation='softmax' ))

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                      optimizer='rmsprop', metrics=['accuracy'])


In [ ]:
model.fit(train_dataset, ) #model.add(tf.keras.layers.LSTM(64))

In [ ]:
model.save('../raw_data/model/model_lstm64')

In [ ]:
model2 = tf.keras.Sequential()

model2.add(tf.keras.layers.Masking(mask_value=-999.))
model2.add(tf.keras.layers.LSTM(128))
# model2.add(tf.keras.layers.LSTM(32))
model2.add(tf.keras.layers.Dense(7,activation='softmax' ))

model2.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                      optimizer='rmsprop', metrics=['accuracy'])

### Q and A

In [ ]:
#visualize data PCA
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
train_examples.shape

In [ ]:
#sample
X = np.array([[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1]])
X.shape

In [ ]:
padded_inputs[0]

In [ ]:
flatted_inputs = [ r.flatten() for r in padded_inputs]

In [ ]:
len(flatted_inputs)

In [ ]:
train_examples_embedded = TSNE(n_components=2).fit_transform(flatted_inputs)

In [ ]:
train_examples_embedded.shape